# Trade network analysis
**Brian Dew (brianwdew@gmail.com)**

`01oil_DOT.ipynb`

Builds a set of bilateral trading relationships for each period between ( and ). The source is the IMF's Direction of Trade Statistics (DOTS), with data retrieved through the JSON RESTful API CompactData method.

Required file:

* c_list.csv - List of countries to keep. Regions are dropped.

---

   

#### Libraries

In [15]:
import requests                                             # For requesting json data from the url
import pandas as pd                                         # pandas dataframes used for convenience
import numpy as np
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data')

#### Define local variables

In [16]:
webserv = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/' # the main URL for the JSON rest API
methodCD = 'CompactData/'                                     # CompactData contains only the data 
methodDS = 'DataStructure/' # This method gives info on the country names, units, and indicator names
series = 'DOT'                                              # International Financial Statistics series

#### Obtain unit multiplier code list

In [17]:
urlDS = webserv+methodDS+series   # url to access IMF datastructure method API
# Request data from IMF JSON RESTful API URL above. Navigate to the code list:
dataDS = requests.get(urlDS).json()['Structure']['CodeLists']['CodeList']#

df = pd.DataFrame(dataDS[0]['Code']).set_index('@value')               # unit codes
unit_codes = {m : df['Annotations'].loc[m]['Annotation'][2]['AnnotationText']['#text'] 
              for m in df.index.values}

#### Collect Data

In [18]:
full_data = {}
country_list = pd.read_csv('c_list.csv', header=None)[0].values
country_list = np.append(country_list, 'NA')
for cty in country_list:
    try:
        url = webserv+methodCD+series+'/A.'+cty+'.TMG_CIF_USD..?startPeriod=2008&endPeriod=2015'
        raw = requests.get(url).json()
        df = pd.DataFrame(raw['CompactData']['DataSet']['Series'])
        df = df[df['@COUNTERPART_AREA'].isin(country_list)].set_index('@COUNTERPART_AREA')
        df['@UNIT_MULT'] = df['@UNIT_MULT'].map(unit_codes) # match unit codes with unit multipliers
        df = df[df['Obs'].apply(lambda x: isinstance(x, list))] # drops empties
        d = {} # temporary dict to save country by country dataframes
        for c in df.index.values: # index values are countries (@REF_AREA) as set above
            d[c] = pd.DataFrame(df.loc[c]['Obs']).rename(columns={'@TIME_PERIOD':'date'})
            # Multiply units by unity multiplier value. 
            d[c]['@OBS_VALUE'] = pd.to_numeric(d[c]['@OBS_VALUE']) * int(df['@UNIT_MULT'][c]) 
        # Concatenate all country rows into one dataframe for each indicator:
        fd = pd.concat(d, axis=0).reset_index().set_index('date')
        fd = fd.rename(columns={'level_0':'j', '@OBS_VALUE': 'x'})
        fd['i'] = cty
        full_data[cty] = fd[['i', 'j', 'x']].sort_index()
    except Exception:                                   # some products don't converge
        pass 

In [19]:
full_data_clean = pd.concat(full_data, axis=0).reset_index().set_index('date')[['i', 'j', 'x']].sort_index()
full_data_clean = full_data_clean[full_data_clean.x != 0]
full_data_clean.to_csv('A_DOTS_all.csv')